In [ ]:
!pip3 install chart-studio 

     |████████████████████████████████| 71kB 2.6MB/s 


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving data_job_new.txt to data_job_new.txt


In [ ]:
%matplotlib inline

import numpy as np
import math
import time
import random
import pickle
import plotly
import chart_studio
import chart_studio.plotly as py
import plotly.graph_objs as go

# Enter your own plotly credentials to play on plots
chart_studio.tools.set_credentials_file(username='', api_key='')

data = []
f = open("data_job_new.txt", 'r')
temp = []
for a in f:
  d = a.split()
  if (len(d) != 0):
    temp.append(d)
  else:
    data.append(temp)
    temp = []
data.append(temp)
temp = []



print(data)
#Матрица возможности выйти на смену
matrixProb = []
for i in data[0]:
  t = []
  for k in i:
    t.append(int(k))
    
  matrixProb.append(t)

global count
#Количество смен
count = len(matrixProb[0])
#Матрица требуемых людей на смену
matrixCount = []
for i in data[1][0]:
  matrixCount.append(int(i))
#Матрица максимальных затрат работников
matrixMaxJob = []
for i in data[3][0]:
  matrixMaxJob.append(int(i))
#Матрица оплаты работы
matrixPayJob = []
for i in data[4][0]:
  matrixPayJob.append(int(i))
print('Матрица возможности выйти на смену', matrixProb)
print('Матрица максимальных затрат сотрудника', matrixMaxJob)
print("Матрица цены смены работника", matrixPayJob)
startResult = []
#Длительность 1 смены
timerVal = 8
for i in range(0,count):
  startResult.append(-1)
print("Матрица стартовых значений позиций смен", startResult)



global lastMatrixFit
lastMatrixFit = [20] * len(matrixMaxJob)

[[['0', '1', '1', '0', '1', '1', '0', '0', '0', '1', '1', '1', '1', '1', '1'], ['0', '0', '1', '1', '1', '1', '0', '0', '0', '1', '1', '1', '1', '0', '1'], ['1', '0', '1', '0', '1', '0', '1', '1', '0', '1', '0', '1', '0', '0', '0']], [['2', '1', '2']], [['0', '0', '1']], [['20', '35', '44']], [['10', '15', '20']]]
Матрица возможности выйти на смену [[0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1], [0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 1], [1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0]]
Матрица максимальных затрат сотрудника [20, 35, 44]
Матрица цены смены работника [10, 15, 20]
Матрица стартовых значений позиций смен [-1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1, -1]


In [ ]:
def validateSolution(solution):
  matrixOverWork = matrixMaxJob.copy()
  for a in solution:
    matrixOverWork[a] -= timerVal
    if (matrixOverWork[a] < 0):
      break
      return False
  lastMatrixFit = matrixOverWork.copy()
  #print(lastMatrixFit)
  return True
def validSolution(count):
  startResult = []
  for i in range(0,count):
    startResult.append(-1)
  temp = generateSolution(matrixSolutions, matrixMaxJob,startResult, count)
  while (not validateSolution(temp[0])):
    temp = generateSolution(matrixSolutions, matrixMaxJob,startResult, count)
  return temp[0]

def softConstain(solution):
  matrixOverWork = matrixMaxJob.copy()
  for a in solution:
    matrixOverWork[a] -= timerVal
  for smena in range(0, len(solution)):
    if (matrixProb[solution[smena]] != 1):
      for worker in range(0, len(matrixProb)):
        if (worker != solution[smena]):
          if (matrixProb[worker][smena] == 1):
            if (matrixOverWork[worker] - timerVal >= 0):
              matrixOverWork[solution[smena]] += timerVal
              solution[smena] = worker;
              matrixOverWork[worker] -= timerVal
              break
  return solution

In [ ]:
def generateSolution(prevSolutions,matrixMaxJob,ad, count):
  #Длительность 1 смены
  timerVal = 8
  #Матрица-определитель количества оставшихся у каждого работника часов выработки до переработки
  matrixJobFit = matrixMaxJob[:]
  i = 0
  result = ad[:]
  d = 0
  while (d < len(result)):
    result[d] = random.randint(0,len(matrixMaxJob)-1)
    d += 1
  return [result, matrixJobFit]
solutions = []
print(generateSolution(solutions,matrixMaxJob, startResult.copy(), count)[0])

def getCost(solutionFactor, matrixPayJob):
  result = 0
  for a in solutionFactor:
        result += matrixPayJob[a]
  return result

#Функция нахождения следующего решения для перехода
def neighbour(matrixSolutions, count):
  res = softConstain(validSolution(count))
  matrixSolutions.append(res)
  return res

#Алгоритм отжига
def E(s):
  return getCost(s, matrixPayJob)
def P(e_new,e_old,T):
  return math.exp(-1 * (e_old - e_new) / T)
def temperature(Targ,i):
  return Targ * 0.1 / i;
def anneal(T_min, T_max, matrixSolutions, maxIter, count):
  Timing = []
  Tseq = []
  Sseq = []
  Fseq=  []
  s0 = neighbour(matrixSolutions, count)
  
  Tinit = random.uniform(T_min,T_max)
  T = Tinit
  s = s0
  i = 1
  while maxIter > i:
      Timing.append(i)
      Sseq.append(s)
      Fseq.append(E(s))
      Tseq.append(T)
      s_new = neighbour(matrixSolutions, count)
      e_old = E(s)
      e_new = E(s_new)
      deltaE = e_new  - e_old
      if (deltaE < 0):
        s = s_new
      else:
        if P(e_old, e_new, T) >= random.random():
          s = s_new
      T = temperature(Tinit,i)
      i += 1
      if T_min > T:
        break
  return [s,Sseq,Tseq,Timing, Fseq]



[0, 1, 0, 0, 1, 1, 2, 1, 1, 0, 2, 0, 2, 2, 2]


In [ ]:
matrixSolutions = []
import plotly.express as px
Tseq = []
testSeq = []
timing = []
l = anneal(0.00001,1000, matrixSolutions, 100, count)
testSeq = l[1]
Tseq = l[2]
timing = l[3]
result = l[0]
Fseq = l[4]
print(lastMatrixFit)
print(result)
import pandas as pd

[20, 20, 20]
[0, 1, 1, 2, 2, 0, 1, 2, 2, 1, 0, 2, 0, 0, 0]


In [ ]:
df = pd.DataFrame( list(zip(timing,Fseq)),columns= ["Iterations", "Solution Cost"])
fig = px.line(df, x="Iterations",  y="Solution Cost", title='Fit_function/time', width=400, height=400)
fig.show()

In [ ]:
df = pd.DataFrame( list(zip(timing,Tseq)),columns= ["Iterations", "Temperature"])
fig = px.line(df, x="Iterations", y="Temperature", title='Temperature/time', width=400, height=400)
fig.show()